In [1]:

# import acquire as a
# from env import github_token, github_username
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import scipy.stats as stats
from scipy.stats import  chi2_contingency
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import ttest_ind
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

import sys
sys.path.append("..")
import acquire as a
from env import github_token, github_username
"""
A module for obtaining repo readme and language data from the github API.
Before using this module, read through it, and follow the instructions marked
TODO.
After doing so, run it like this:
    python acquire.py
To create the `data.json` file that contains the data.
"""
import json
from typing import Dict, List, Optional, Union, cast



**Get data**

In [14]:
# repo = 'python/cpython' # repository identification
# authentications
headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

REPO_NAME = []
for i in range(1, 10):
    # url to python repos
    python_url= f'https://github.com/search?o=desc&q=stars%3A%3E1+language%3APython&s=forks&type=Repositories&p=1&l=python={i}'

    # get json rescponce
    res = a.github_api_request(python_url)
    # repo_content = a.get_repo_contents(repo)
    temp_name = [REPO_NAME.append(x['hl_name']) for x in res['payload']['results']]


In [5]:
url_link = []
readme_con = []
repo_names = []
rst_file= []
for page_repo in REPO_NAME:
    repo_content = a.get_repo_contents(page_repo)
    # locate the Readme.rst file link
    for ele in range(len(repo_content)):
        link = repo_content[ele]["html_url"]
        match = re.search(f"README", link)
        if match:
            rst_file = link
            break
    # get the readme request
    readme_res = requests.get(rst_file)
    
    if readme_res.status_code == 200:
        # find read me content
        soup = BeautifulSoup(readme_res.text, 'html.parser')
        readme_content = soup.get_text()

        # extract noisy charactures from the content
        pattern = r'"richText":"(.*?)"\s*,\s*"renderedFileInfo"'
        matches = re.findall(pattern, readme_content)
        extracted_content = matches[0].replace("\\n","")
        
        # get repo language
        repo_name = acquire.get_repo_language(page_repo)

        # url and readme content of all repo
        url_link.append(rst_file)
        readme_con.append(extracted_content)
        repo_names.append(repo_name)
    else:
        print("Failed to connect...!")

In [ ]:
results = pd.DataFrame(REPO_NAME).assign(url = url_link, readme_content = readme_con)
results

In [ ]:
acquire.get